In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

# Load match data and convert from GMT+8 to UTC
df_match = pd.read_csv("all_acm_matches_full.csv", parse_dates=["Datetime"])
df_match["Datetime"] = df_match["Datetime"].dt.tz_localize("Asia/Singapore").dt.tz_convert("UTC")
df_match["Datetime"] = df_match["Datetime"].dt.tz_localize(None)

# Load candle data
df_candle = pd.read_csv("acm_usdt_15m.csv", parse_dates=["Open Time"])
df_candle.set_index("Open Time", inplace=True)
df_candle = df_candle[["Open", "High", "Low", "Close", "Volume"]].astype(float)

# Get the latest 5 matches
latest_matches = df_match.sort_values(by="Datetime", ascending=False).head(20)
day_window = 1

# Generate one plot per match
for idx, main_row in latest_matches.iterrows():
    match_time = main_row["Datetime"]
    window_start = match_time - timedelta(days=day_window)
    window_end = match_time + timedelta(days=day_window)

    df_window = df_candle[(df_candle.index >= window_start) & (df_candle.index <= window_end)]
    if df_window.empty:
        continue

    # Filter all matches within this window
    matches_in_window = df_match[df_match["Datetime"].between(window_start, window_end)]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

    # OHLC lines
    ax1.plot(df_window.index, df_window["Open"], label="Open", linewidth=0.5)
    ax1.plot(df_window.index, df_window["Close"], label="Close", linewidth=0.5)
    ax1.plot(df_window.index, df_window["High"], label="High", linewidth=0.5)
    ax1.plot(df_window.index, df_window["Low"], label="Low", linewidth=0.5)

    # Annotate and draw ±5% lines for each match in the window
    for _, row in matches_in_window.iterrows():
        if row["Datetime"] not in df_window.index:
            continue

        open_price = df_window.loc[row["Datetime"], "Open"]
        y = df_window.loc[row["Datetime"], "Close"]
        color = "green" if row["Result"] == "Win" else "red" if row["Result"] == "Loss" else "blue"
        marker = "^" if row["Result"] == "Win" else "v"
        label = f"{row['League']} MD{row['Matchday']} ({row['Home/Away']})"

        # Annotation
        ax1.plot(row["Datetime"], y, marker=marker, color=color, markersize=10)
        ax1.text(row["Datetime"], y, label, fontsize=8, rotation=90, va='bottom', ha='center')
        if row["Datetime"] != match_time:
            continue
        # Horizontal ±5% lines
        plus_5 = open_price * 1.03
        minus_5 = open_price * 0.97
        ax1.axhline(y=plus_5, color=color, linestyle='-', linewidth=2.5)
        ax1.axhline(y=minus_5, color=color, linestyle='-', linewidth=2.5)

    # Vertical lines every hour
    hour_ticks = pd.date_range(start=window_start, end=window_end, freq='1h')
    for tick in hour_ticks:
        ax1.axvline(x=tick, color='gray', linewidth=0.3, linestyle='--')
        ax2.axvline(x=tick, color='gray', linewidth=0.3, linestyle='--')

    ax1.set_title(f"ACM/USDT with All Matches in ±{day_window} Days of {match_time.strftime('%Y-%m-%d %H:%M')}")
    ax1.set_ylabel("Price (USDT)")
    ax1.legend()
    ax1.grid(True)

    # Volume plot
    ax2.plot(df_window.index, df_window["Volume"], label="Volume", color="gray", linewidth=2.5)
    ax2.set_yscale("log")
    ax2.axhline(y=40000)
    ax2.axhline(y=200000)
    ax2.set_ylabel("Volume")
    ax2.set_xlabel("Time (UTC)")
    ax2.grid(True)
    ax2.legend()

    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d\n%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()

    plot_file = f"3_1_3_acm_{day_window}days_match_{match_time.strftime('%Y%m%d_%H%M')}_allmatches.png"
    plt.savefig(plot_file)
    plt.close()
    print(f"Saved: {plot_file}")


Saved: 3_1_3_acm_1days_match_20250405_1845_allmatches.png
Saved: 3_1_3_acm_1days_match_20250402_1900_allmatches.png
Saved: 3_1_3_acm_1days_match_20250330_1845_allmatches.png
Saved: 3_1_3_acm_1days_match_20250315_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250308_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250302_1945_allmatches.png
Saved: 3_1_3_acm_1days_match_20250227_1945_allmatches.png
Saved: 3_1_3_acm_1days_match_20250222_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250218_1745_allmatches.png
Saved: 3_1_3_acm_1days_match_20250215_1945_allmatches.png


/var/folders/wn/4k27w4bd5yz_qc4239ww31m00000gq/T/ipykernel_65712/3940099379.py:86: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


Saved: 3_1_3_acm_1days_match_20250212_2000_allmatches.png
Saved: 3_1_3_acm_1days_match_20250208_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250205_2000_allmatches.png
Saved: 3_1_3_acm_1days_match_20250202_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250129_2000_allmatches.png
Saved: 3_1_3_acm_1days_match_20250126_1130_allmatches.png
Saved: 3_1_3_acm_1days_match_20250122_2000_allmatches.png
Saved: 3_1_3_acm_1days_match_20250118_1700_allmatches.png
Saved: 3_1_3_acm_1days_match_20250114_1730_allmatches.png
Saved: 3_1_3_acm_1days_match_20250111_1945_allmatches.png


In [16]:
1+1

2